# Strategy

Prior to diving into the strategy itself, we will generate the token universe based on the outlined conditions. For the sake of simplicity we will refer to tokens that are traded on Binance.

In [10]:
from velodata import lib as velo
from dotenv import load_dotenv

import os
import polars as pl

load_dotenv()

day_in_ms = 1000 * 60 * 60 * 24
client = velo.client(os.environ.get('VELO_API_KEY'))

futs = client.get_products('futures')

# filter based on futures on binance
filtered_instruments = [f['coin'] for f in futs if f['exchange'] == 'binance-futures']

# next get the top 30 based by 24h volume
volumes = client.get_rows({
  'type': 'futures',
  'exchanges': ['binance-futures'],
  'coins': filtered_instruments,
  'begin': client.timestamp() - day_in_ms,
  'end': client.timestamp(),
  'columns': ['dollar_volume'],
  'resolution': 60 * 24
})

df_volumes =  (
  pl.DataFrame(volumes).with_columns(
        pl.from_epoch("time", time_unit="ms").alias("date")
  )
  .group_by("coin")
  .first()
  ).sort("dollar_volume", descending=True)

df_volumes


coin,exchange,product,time,dollar_volume,date
str,str,str,i64,f64,datetime[ms]
"""BTC""","""binance-futures""","""BTCUSDT""",1723075200000,4.5036e9,2024-08-08 00:00:00
"""ETH""","""binance-futures""","""ETHUSDT""",1723075200000,1.6371e9,2024-08-08 00:00:00
"""SOL""","""binance-futures""","""SOLUSDT""",1723075200000,5.8075e8,2024-08-08 00:00:00
"""WIF""","""binance-futures""","""WIFUSDT""",1723075200000,1.7776e8,2024-08-08 00:00:00
"""1000PEPE""","""binance-futures""","""1000PEPEUSDT""",1723075200000,1.6093e8,2024-08-08 00:00:00
…,…,…,…,…,…
"""OXT""","""binance-futures""","""OXTUSDT""",1723075200000,378893.437,2024-08-08 00:00:00
"""RLC""","""binance-futures""","""RLCUSDT""",1723075200000,355041.811,2024-08-08 00:00:00
"""BNT""","""binance-futures""","""BNTUSDT""",1723075200000,349837.327,2024-08-08 00:00:00


In [3]:

# get daily candles for the past two years
params = {
      'type': 'futures',
      'columns': ['open_price', 'close_price'],
      'exchanges': ['binance-futures'],
      'products': ['ETHUSDT'],
      'begin': client.timestamp() - day_in_ms * 90,
      'end': client.timestamp(),
      'resolution': 1 * 60 * 24
    }
    
# returns dataframe
# batches = client.batch_rows(params)  

# for df in client.stream_rows(batches):
#   print(df)
  